In [1]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import json



from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from spdnetwork.optimizers import  MixOptimizer 
from spdnetwork.nn import LogEig
from Utils import get_fold_of_data
from DatasetManagement import DatasetManagement
from Models import   Contrastive_CB3

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
i=1


train_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsTriplet//fold_{i}/train/embeddings_train.pt'
train_labels_path = f'/data/ProstateContrastiveSPD/embeddingsTriplet//fold_{i}/train/labels_train.pt'

train_embeddings = torch.load(train_embeddings_path)
train_labels = torch.load(train_labels_path)

val_embeddings_path = f'/data/ProstateContrastiveSPD/embeddings/fold_{i}/val/embeddings_val.pt'
val_labels_path = f'/data/ProstateContrastiveSPD/embeddings/fold_{i}/val/labels_val.pt'

val_embeddings = torch.load(val_embeddings_path)
val_labels = torch.load(val_labels_path)



print(train_embeddings.shape)
print(train_labels.shape)
print('')
print(val_embeddings.shape)
print(val_labels.shape)



torch.Size([1035, 4560])
torch.Size([1035, 1])

torch.Size([260, 4560])
torch.Size([260, 1])


# Logistic Regression

In [8]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc

def train_and_evaluate(embeddings_train_np, labels_train_np, embeddings_val_np, labels_val_np):
    # Entrenar el modelo de regresión logística
    model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
    model.fit(embeddings_train_np, labels_train_np)

    # Predicciones en el conjunto de validación
    predictions = model.predict(embeddings_val_np)
    predictions_proba = model.predict_proba(embeddings_val_np)[:, 1]

    # Calcular la exactitud
    accuracy = accuracy_score(labels_val_np, predictions)
    auc_roc = roc_auc_score(labels_val_np, predictions_proba)
    precisions, recalls, _ = precision_recall_curve(labels_val_np, predictions_proba)
    auc_pr = auc(recalls, precisions)

    return accuracy, auc_roc, auc_pr

percentages = [0.2, 0.4, 0.6, 0.8]
metrics = {p: {'accuracy': [], 'auc_roc': [], 'auc_pr': []} for p in percentages}

for i in range(5):
    # Cargar embeddings y etiquetas de entrenamiento
    embeddings_train = torch.load(f'/data/ProstateContrastiveSPD/embeddings/fold_{i+1}/train/embeddings_train.pt')
    labels_train = torch.load(f'/data/ProstateContrastiveSPD/embeddings/fold_{i+1}/train/labels_train.pt')

    # Cargar embeddings y etiquetas de validación
    embeddings_val = torch.load(f'/data/ProstateContrastiveSPD/embeddings/fold_{i+1}/val/embeddings_val.pt')
    labels_val = torch.load(f'/data/ProstateContrastiveSPD/embeddings/fold_{i+1}/val/labels_val.pt')

    # Convertir a numpy arrays
    embeddings_train_np = embeddings_train.cpu().numpy()
    labels_train_np = labels_train.cpu().numpy()

    embeddings_val_np = embeddings_val.cpu().numpy()
    labels_val_np = labels_val.cpu().numpy()

    # Entrenar y evaluar para cada porcentaje
    for p in percentages:
        n_samples = int(len(embeddings_train_np) * p)
        selected_indices = np.random.choice(len(embeddings_train_np), n_samples, replace=False)

        embeddings_subset = embeddings_train_np[selected_indices]
        labels_subset = labels_train_np[selected_indices]

        accuracy, auc_roc, auc_pr = train_and_evaluate(embeddings_subset, labels_subset, embeddings_val_np, labels_val_np)

        metrics[p]['accuracy'].append(accuracy)
        metrics[p]['auc_roc'].append(auc_roc)
        metrics[p]['auc_pr'].append(auc_pr)

# Calcular y mostrar promedios y desviaciones estándar
for p in percentages:
    print(f"\nPorcentaje de entrenamiento: {int(p*100)}%")
    print(f"Promedio de AUC-ROC: {np.mean(metrics[p]['auc_roc']):.4f}")
    print(f"Desviación estándar de AUC-ROC: {np.std(metrics[p]['auc_roc']):.4f}")
    print(f"Promedio de Accuracy: {np.mean(metrics[p]['accuracy']):.4f}")
    print(f"Desviación estándar de Accuracy: {np.std(metrics[p]['accuracy']):.4f}")
    print(f"Promedio de AUC-PR: {np.mean(metrics[p]['auc_pr']):.4f}")
    print(f"Desviación estándar de AUC-PR: {np.std(metrics[p]['auc_pr']):.4f}")


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam


Porcentaje de entrenamiento: 20%
Promedio de AUC-ROC: 0.8931
Desviación estándar de AUC-ROC: 0.0159
Promedio de Accuracy: 0.9011
Desviación estándar de Accuracy: 0.0038
Promedio de AUC-PR: 0.7479
Desviación estándar de AUC-PR: 0.0426

Porcentaje de entrenamiento: 40%
Promedio de AUC-ROC: 0.9061
Desviación estándar de AUC-ROC: 0.0208
Promedio de Accuracy: 0.9066
Desviación estándar de Accuracy: 0.0048
Promedio de AUC-PR: 0.7647
Desviación estándar de AUC-PR: 0.0182

Porcentaje de entrenamiento: 60%
Promedio de AUC-ROC: 0.9184
Desviación estándar de AUC-ROC: 0.0196
Promedio de Accuracy: 0.9237
Desviación estándar de Accuracy: 0.0155
Promedio de AUC-PR: 0.7998
Desviación estándar de AUC-PR: 0.0303

Porcentaje de entrenamiento: 80%
Promedio de AUC-ROC: 0.9206
Desviación estándar de AUC-ROC: 0.0150
Promedio de Accuracy: 0.9150
Desviación estándar de Accuracy: 0.0035
Promedio de AUC-PR: 0.8078
Desviación estándar de AUC-PR: 0.0284


# SVM

In [14]:
import torch
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

array_auc = []
array_accuracy = []
array_auc_pr = []

for i in range(5):
    # Cargar embeddings y etiquetas de entrenamiento
    embeddings_train = torch.load(f'/data/ProstateContrastiveSPD/embeddingsTriplet/fold_{i+1}/train/embeddings_train.pt')
    labels_train = torch.load(f'/data/ProstateContrastiveSPD/embeddingsTriplet/fold_{i+1}/train/labels_train.pt')

    # Cargar embeddings y etiquetas de validación
    embeddings_val = torch.load(f'/data/ProstateContrastiveSPD/embeddingsTriplet/fold_{i+1}/val/embeddings_val.pt')
    labels_val = torch.load(f'/data/ProstateContrastiveSPD/embeddingsTriplet/fold_{i+1}/val/labels_val.pt')

    # Convertir a numpy arrays
    embeddings_train_np = embeddings_train.cpu().numpy()
    labels_train_np = labels_train.cpu().numpy()

    embeddings_val_np = embeddings_val.cpu().numpy()
    labels_val_np = labels_val.cpu().numpy()

    # Entrenar el modelo de SVM
    model = make_pipeline(StandardScaler(), SVC(probability=True, random_state=42))
    model.fit(embeddings_train_np, labels_train_np)

    # Predicciones en el conjunto de validación
    predictions = model.predict(embeddings_val_np)
    predictions_proba = model.predict_proba(embeddings_val_np)[:, 1]

    # Calcular la exactitud
    accuracy = accuracy_score(labels_val_np, predictions)
    print(f'Accuracy: {accuracy:.4f}')
    array_accuracy.append(accuracy)

    # Calcular el AUC-ROC
    auc_roc = roc_auc_score(labels_val_np, predictions_proba)
    print(f'AUC-ROC: {auc_roc:.4f}')
    array_auc.append(auc_roc)

    # Calcular el AUC-PR
    auc_pr = average_precision_score(labels_val_np, predictions_proba)
    print(f'AUC-PR: {auc_pr:.4f}')
    array_auc_pr.append(auc_pr)

print(f"Promedio de AUC-ROC: {np.mean(array_auc)}")
print(f"Desviación de AUC-ROC: {np.std(array_auc)}")
print("")
print(f"Promedio de Accuracy: {np.mean(array_accuracy)}")
print(f"Desviación de Accuracy: {np.std(array_accuracy)}")
print("")
print(f"Promedio de AUC-PR: {np.mean(array_auc_pr)}")
print(f"Desviación de AUC-PR: {np.std(array_auc_pr)}")


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9077
AUC-ROC: 0.9146
AUC-PR: 0.8234


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9242
AUC-ROC: 0.9127
AUC-PR: 0.8195


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9134
AUC-ROC: 0.9085
AUC-PR: 0.7733


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9094
AUC-ROC: 0.9501
AUC-PR: 0.8529


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9286
AUC-ROC: 0.9510
AUC-PR: 0.8731
Promedio de AUC-ROC: 0.9273877483909668
Desviación de AUC-ROC: 0.019038727771807738

Promedio de Accuracy: 0.9166651899083931
Desviación de Accuracy: 0.008279257096420918

Promedio de AUC-PR: 0.8284379196775014
Desviación de AUC-PR: 0.03388785960418474


1
